Exercise 8


In [1]:
from sklearn.datasets import fetch_openml

In [2]:
data = fetch_openml(name="mnist_784", version = "active")


In [3]:
X = data["data"]
y = data["target"]

In [4]:
#split instances for training and test/val
from sklearn.model_selection import train_test_split
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=20000)

In [5]:
#Split instances for test and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size = 10000)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC


In [ ]:
#Create our models
Logistic_clf = LogisticRegression()
forest_clf = RandomForestClassifier(n_estimators=150, max_depth=5, min_samples_split=100)
Extra_tree = ExtraTreesClassifier(n_estimators=150, max_depth=5, min_samples_split=100)
svc = SVC(probability=True)
linear_svc = LinearSVC()


In [ ]:
#Logistic_clf.fit(X_train,y_train)
forest_clf.fit(X_train,y_train)
Extra_tree.fit(X_train,y_train)
svc.fit(X_train,y_train)
#linear_svc.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
forest_predictions = forest_clf.predict(X_val)
Extra_tree_predictions = Extra_tree.predict(X_val)
svc_predictions = svc.predict(X_val)

In [ ]:
from sklearn.metrics import accuracy_score
print(f"Accuracy for forest: {accuracy_score(y_val,forest_predictions)}")
print(f"Accuracy for Extra-trees: {accuracy_score(y_val,Extra_tree_predictions)}")
print(f"Accuracy for svc: {accuracy_score(y_val,svc_predictions)}")

Accuracy for forest: 0.8575
Accuracy for Extra-trees: 0.8465
Accuracy for svc: 0.9784


In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[("forest", forest_clf), ("Extra-trees",Extra_tree),("SVC" ,svc)],
                              voting='hard'
                              )

In [ ]:
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('forest',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=5,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=100,
                                                     min_weight_fraction_leaf=0.0,
     

In [ ]:
voting_predictions = voting_clf.predict(X_val)


In [ ]:
voting_clf.score(X_val, y_val)

0.8891

In [ ]:
print(f"Accuracy for hard voting: {accuracy_score(y_val,voting_predictions)}")

Accuracy for hard voting: 0.8891


In [ ]:
#Try to change to soft voting
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

0.9745

# **Exercise 9**
## * Let's Create a Blender!


In [7]:
# Let's save our trained models

import joblib
from google.colab import files
def model_saver_and_downloader(model, name):
  joblib.dump(model, f"{name}.pkl")
  files.download(f"{name}.pkl")

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
#model_saver_and_downloader(forest_clf, "forest_clf")
#model_saver_and_downloader(Extra_tree, "Extra_tree")
#model_saver_and_downloader(svc, "svc")
#model_saver_and_downloader(voting_clf, "voting_clf")

#Calling the models and using them
forest_clf_saved = joblib.load('/content/gdrive/My Drive/forest_clf.pkl') #this calls our trained model
Extra_tree_saved = joblib.load("/content/gdrive/My Drive/Extra_tree.pkl")
#svc_saved = joblib.load("/content/gdrive/My Drive/svc.pkl")
voting_clf_saved = joblib.load("/content/gdrive/My Drive/voting_clf.pkl")
forest_predictions = forest_clf_saved.predict(X_val)
Extra_tree_predictions = Extra_tree_saved.predict(X_val)

In [9]:
#Let's Create a Blender!
forest_clf_blender = RandomForestClassifier()
import pandas as pd




In [10]:

#Create a new training set with the validation prediction of all the predictors
first_layer_predictions_training_set =  pd.DataFrame()
first_layer_predictions_training_set['forest_predictions'] = forest_clf_saved.predict(X_val)
first_layer_predictions_training_set['Extra_trees_predictions'] = Extra_tree_saved.predict(X_val)
#first_layer_predictions_training_set['SVC_predictions'] = linear_svc_predictions
first_layer_predictions_training_set['labels'] = y_val


In [11]:
first_layer_predictions_training_set

,forest_predictions,Extra_trees_predictions,labels
0,5,8,6
1,4,4,4
2,5,5,5
3,0,0,0
4,4,4,4
...,...,...,...
9995,0,0,6
9996,8,2,2
9997,3,3,3
9998,3,3,3


In [12]:
#Split the new dataset into X and y
X_blender = first_layer_predictions_training_set.to_numpy()[:, :2]
y_blender = first_layer_predictions_training_set['labels']
forest_clf_blender.fit(X_blender,y_blender)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
#Test the blender with the test_set
#create predictions for the test test and create a new test set for predictions 
from sklearn.metrics import accuracy_score
#Create the input to the blender
blender_test_set  =  pd.DataFrame()
blender_test_set['forest_predictions'] = forest_clf_saved.predict(X_test)
blender_test_set['Extra_trees_predictions'] = Extra_tree_saved.predict(X_test)
#blender_test_set['SVC_predictions'] = svc_test_predictions


blender_predictions = forest_clf_blender.predict(blender_test_set)


#Check accuracy

print(f"Accuracy for forest: {accuracy_score(y_val,forest_predictions)}")
print(f"Accuracy for Extra-trees: {accuracy_score(y_val,Extra_tree_predictions)}")
print(f"Accuracy for blender: {accuracy_score(y_test, blender_predictions)}")

Accuracy for forest: 0.8623
Accuracy for Extra-trees: 0.8504
Accuracy for blender: 0.8698
